# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/arabic_dialogue_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [ ]:
df = load_dataset("CUTD/arabic_dialogue_df", split="train[:30%]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df

Dataset({
    features: ['Unnamed: 0', 'dialogue', 'summary'],
    num_rows: 4500
})

In [ ]:
df["dialogue"]

['أوبرا: خمن ما أعطانيه جو ليومبي\n نايجل: الجوارب؟\n ميني: هزة الجماع؟\n أوبرا: <file_photo>\n أوبرا: يا رفاق هم الأسوأ\n نايجل: لول\n ميني: كنت على حق نوعا ما\n أوبرا: أردت أن أذهب إلى هناك بشدة\n نايجل: إذن اشترى لك الطعام؟\n أوبرا: إنه ليس مجرد طعام\n أوبرا: هذه هي الحلويات المفضلة لدي من تركيا\n ميني: لا أحصل على حبك للطعام ولكن كل ما يجعلك سعيدًا\n نايجل: ألا تعاني من حساسية من المكسرات؟\n أوبرا: أنا حساسية من الجوز والكاجو ، لذا فلا بأس\n ميني: تبدو لطيفة جدا\n نايجل: حسنًا\n أوبرا: لديها أيضًا بتلات وردية\n نايجل: تبدو لذيذة',
 'إيليا: مهلا ، في طريقي إلى المكتب.هل يمكنك أن ترسل لي هذا جدول الأعمال لهذا اليوم؟\n بريت: بالتأكيد.إنه جاهز فقط\n إيليا: شكرا.هل كيتلين هناك حتى الآن؟\n بريت: لا ، ليس بعد\n إيليا: حسنًا ، فقط أتساءل.لدينا مكالمة جماعية في التاسعة.\n بريت: هل هذا مع مؤسسة الكرم؟\n إيليا: نعم ، إنهم يفكرون في طلب المنح لدينا ، لكنهم أرادوا إجراء مقابلة معنا أولاً\n بريت: حسنًا ، حظًا سعيدًا في ذلك\n إيليا: شكرا.\n بريت: أرسل للتو جدول أعمال.\n إيليا: حسنًا.سأخبرك عندما 

In [ ]:
df["summary"]

['حصلت أوبرا على حلوياتها المفضلة من تركيا لعيد ميلادها من جو.',
 'بريت يرسل إيليا جدول أعمال اليوم.هناك مكالمة جماعية في التاسعة مع مؤسسة الكرم فيما يتعلق بطلب المنحة.تم تأجيل حفلة عيد الميلاد إلى الخميس.',
 'يعتقد الشخص 2 أن شركة الشخص 1 هي الشركة الرائدة في صناعة الكمبيوتر من أجل حصتها ، وتفوقها في المبيعات والتنوع في الإنتاج.',
 'خرج باتريشيا وكلوي الليلة الماضية.أحب باتريشيا رجل وحصل على رقمه.ومع ذلك ، فهي ليست متأكدة مما إذا كانت ستتصل به.كلوي يعتقد أنه وسيم.',
 'الشخص 1 يسأل الشخص 2 عن يوم زرع الأشجار.الشخص 1 مرتبك لأن الشخص 2 يقول إن زراعة الأشجار وقطع الأشجار تجعل حياتنا أفضل.',
 'تصل جين إلى محطة القطار في الساعة 5 مساءً.تريد جين أن تأخذ سيارة أجرة ولكن جون يقدم مصعد.',
 'تم إلغاء رحلة الشخص 1 وهو منزعج ، لذلك يساعده الشخص 2 في حجز رحلة أخرى ويمنحه خصمًا.',
 'سُرقت دراجة راي من الشارع التاسع أمس على الرغم من تأمينها.لقد كانت دراجة مخصصة حتى الآن راي في القليل من الكبرى.يريد راي نشر ملصقات في المنطقة حتى يخافوا ويتركونها في مكان ما.كان لوقا واحد سرقت الشهر الماضي من نفس المنطق

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('UBC-NLP/AraT5-base')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [ ]:
prefix = "summarize: "

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [ ]:
# to apply preprocessing in all dataset..
df_tokenized = df.map(preprocess_function, batched=True)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='UBC-NLP/AraT5-base', return_tensors="tf")

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained('UBC-NLP/AraT5-base')

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at UBC-NLP/AraT5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

#### split

In [ ]:
train_size = int(0.8 * len(df_tokenized))
test_size = int(0.2 * len(df_tokenized))


df_tokenized = {"train": df_tokenized.select(range(train_size)),
                     "test": df_tokenized.select(range(test_size))}

In [ ]:
df_tokenized

{'train': Dataset({
     features: ['Unnamed: 0', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 3600
 }),
 'test': Dataset({
     features: ['Unnamed: 0', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 900
 })}

In [ ]:
train_size

3600

In [ ]:
test_size

900

In [ ]:
# 2**2 = 4
batch_size= 2

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    df_tokenized["train"],
    shuffle=True, # عشان تتقللب البيانات ويصير المودل مايتعلم على باترنز معينه ويحفظها
    batch_size=batch_size,
    collate_fn=data_collator, # for padding
)

tf_test_set = model.prepare_tf_dataset(
    df_tokenized["test"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

#### parameters

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [ ]:
# to trian Seq2SeqTrainer
model.compile(optimizer=optimizer) # optimizer is defined before

## Step 8: Fine-tune the Model

Train the model using the specified arguments and dataset.

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)

1800/1800 [==============================] - 621s 314ms/step - loss: 15.0776 - val_loss: 8.2436


#### model save

In [ ]:
model.save_pretrained("Summaraization_model")
tokenizer.save_pretrained("/content/mySummaraization_model")

('/content/mySummaraization_model/tokenizer_config.json',
 '/content/mySummaraization_model/special_tokens_map.json',
 '/content/mySummaraization_model/spiece.model',
 '/content/mySummaraization_model/added_tokens.json',
 '/content/mySummaraization_model/tokenizer.json')

## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [ ]:
from transformers import pipeline

In [ ]:
text = '''الشخص 1: هل تعرف أي شخص خضع لعملية جراحية تجميلية؟\n  الشخص 2: تقصد مثل تكبير الثدي؟لا أعتقد أن هذا ضروري حقًا.هذه ليست الطريقة التي يجب أن تظل بها المرأة جذابة.\n  الشخص 1: لماذا لا؟لماذا لا يمكننا العودة إلى الطريقة التي كانت بها الأمور؟\n  الشخص 2: لأن...لا أعرف.لكن ، كوني ، لن تفعل ذلك ، أليس كذلك؟\n  الشخص 1: رتق حق أود!\n  الشخص 2: ربما يكون المكياج كافيًا؟\n  الشخص 1: هيا ، ليز.كن متفتح الذهن!انظر إلى عيني ، فهي منتفخة.وأكياس عيني ، فهي مظلمة وفضفاضة ومليئة بالخطوط والدوائر.آمل حقًا أن تتمكن الجراحة من مسح المشاعر المتعبة.\n  الشخص 2: ولكن ماذا يعتقد الآخرون؟\n  الشخص 1: لا يهمني ما يعتقده الآخرون.أنا مستعد للرفع.'''

In [ ]:
model = '/content/Summaraization_model'
tokenizer = '/content/mySummaraization_model'

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
summarizer(text)

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/Summaraization_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model w

[{'summary_text': 'شنطة.'}]